### Configuration et préparation des données

In [ ]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [ ]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

# Partie 6: Multi-modèles et flexibilité

## 5.1 Introduction: pourquoi utiliser plusieurs modèles?

DSPy offre une **abstraction puissante** : votre code reste le même quel que soit le modèle utilisé. Vous pouvez :

1. **Changer de fournisseur** facilement (Ollama → OpenAI → Anthropic)
2. **Comparer les performances** de différents modèles
3. **Créer des architectures hybrides** (modèle rapide pour catégorie, modèle précis pour priorité)
4. **Optimiser coût vs performance**

### Avantages du multi-modèles

- 🔄 **Flexibilité** : Pas de vendor lock-in
- 💰 **Optimisation des coûts** : Modèle gratuit (Ollama) pour dev, modèle payant pour prod
- 🎯 **Performance** : Choisir le meilleur modèle pour chaque tâche
- 🧪 **Expérimentation** : Tester facilement différents modèles

### Ce que nous allons voir

1. Configurer différents fournisseurs (Ollama, OpenAI, Anthropic)
2. Comparer les performances de différents modèles
3. Créer des architectures hybrides
4. Gérer les clés API de manière sécurisée

## 5.2 Configuration de différents fournisseurs

### 5.2.1 Ollama (local, gratuit)

Ollama permet d'exécuter des modèles **localement** sans API key ni coûts.

**Modèles recommandés** :
- `llama3.1:8b` - Équilibré, bon pour la plupart des tâches (4.7 GB)
- `mistral:7b` - Rapide, bon pour les tâches simples (4.1 GB)
- `qwen2.5:7b` - Haute qualité, excellent pour les tâches complexes (4.7 GB)
- `gemma2:9b` - Alternative de Google, très performant (5.4 GB)

**Configuration** :

In [ ]:
# Configuration Ollama
lm_ollama_llama = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

lm_ollama_mistral = dspy.LM(
    model='ollama_chat/mistral:7b',
    api_base='http://localhost:11434',
    temperature=0.3
)

lm_ollama_qwen = dspy.LM(
    model='ollama_chat/qwen2.5:7b',
    api_base='http://localhost:11434',
    temperature=0.3
)

print("✅ Modèles Ollama configurés:")
print("   - llama3.1:8b")
print("   - mistral:7b")
print("   - qwen2.5:7b")

### 5.2.2 OpenAI (API, payant)

OpenAI propose des modèles très performants via API.

**Modèles recommandés** :
- `gpt-4o-mini` - Rapide et économique, bon rapport qualité/prix
- `gpt-4o` - Haute performance, multimodal
- `gpt-4-turbo` - Équilibré performance/coût

**Configuration** :

In [ ]:
import os

# Configuration OpenAI (nécessite OPENAI_API_KEY dans l'environnement)
if os.getenv('OPENAI_API_KEY'):
    lm_openai_mini = dspy.LM(
        model='openai/gpt-4o-mini',
        temperature=0.3
    )
    
    lm_openai_4o = dspy.LM(
        model='openai/gpt-4o',
        temperature=0.3
    )
    
    print("✅ Modèles OpenAI configurés:")
    print("   - gpt-4o-mini")
    print("   - gpt-4o")
else:
    print("⚠️ OPENAI_API_KEY non définie - modèles OpenAI non disponibles")
    print("   Pour utiliser OpenAI, définissez la variable d'environnement:")
    print("   export OPENAI_API_KEY='votre-clé-api'")
    lm_openai_mini = None
    lm_openai_4o = None

### 5.2.3 Anthropic (API, payant)

Anthropic propose les modèles Claude, connus pour leur qualité et leur sécurité.

**Modèles recommandés** :
- `claude-3-5-haiku-20241022` - Rapide et économique
- `claude-3-5-sonnet-20241022` - Équilibré, excellent pour la plupart des tâches
- `claude-3-opus-20240229` - Maximum de performance

**Configuration** :

In [ ]:
# Configuration Anthropic (nécessite ANTHROPIC_API_KEY dans l'environnement)
if os.getenv('ANTHROPIC_API_KEY'):
    lm_claude_haiku = dspy.LM(
        model='anthropic/claude-3-5-haiku-20241022',
        temperature=0.3
    )
    
    lm_claude_sonnet = dspy.LM(
        model='anthropic/claude-3-5-sonnet-20241022',
        temperature=0.3
    )
    
    print("✅ Modèles Anthropic configurés:")
    print("   - claude-3-5-haiku")
    print("   - claude-3-5-sonnet")
else:
    print("⚠️ ANTHROPIC_API_KEY non définie - modèles Anthropic non disponibles")
    print("   Pour utiliser Anthropic, définissez la variable d'environnement:")
    print("   export ANTHROPIC_API_KEY='votre-clé-api'")
    lm_claude_haiku = None
    lm_claude_sonnet = None

## 5.3 Comparer les performances de différents modèles

Maintenant que nous avons configuré plusieurs modèles, comparons leurs performances sur notre tâche de classification de tickets IT.

### Fonction de benchmarking

In [ ]:
import time

def benchmark_model(lm, model_name, examples, metric):
    """
    Évalue un modèle sur un ensemble d'exemples
    
    Args:
        lm: Le language model DSPy
        model_name: Nom du modèle (pour affichage)
        examples: Liste d'exemples de validation
        metric: Fonction de métrique
    
    Returns:
        dict avec score et temps d'exécution
    """
    # Configurer DSPy avec ce modèle
    dspy.configure(lm=lm)
    
    # Créer un classifier avec ce modèle
    classifier = SimpleTicketClassifier()
    
    # Mesurer le temps
    start_time = time.time()
    
    # Évaluer
    total_score = 0
    for example in examples:
        prediction = classifier(ticket=example['ticket'])
        score = metric(example, prediction)
        total_score += score
    
    end_time = time.time()
    
    # Calculer les résultats
    avg_score = total_score / len(examples)
    elapsed_time = end_time - start_time
    
    return {
        'model': model_name,
        'score': avg_score,
        'time': elapsed_time
    }

print("✅ Fonction de benchmarking définie")

### Comparaison des modèles Ollama

Comparons les 3 modèles Ollama locaux :

In [ ]:
print("🔍 Comparaison des modèles Ollama...")
print("⏰ Cela va prendre quelques minutes\n")

results = []

# Benchmarker llama3.1:8b
print("1/3 Évaluation de llama3.1:8b...")
result_llama = benchmark_model(lm_ollama_llama, 'llama3.1:8b', valset, exact_match_metric)
results.append(result_llama)
print(f"   Score: {result_llama['score']:.2%} | Temps: {result_llama['time']:.1f}s\n")

# Benchmarker mistral:7b
print("2/3 Évaluation de mistral:7b...")
result_mistral = benchmark_model(lm_ollama_mistral, 'mistral:7b', valset, exact_match_metric)
results.append(result_mistral)
print(f"   Score: {result_mistral['score']:.2%} | Temps: {result_mistral['time']:.1f}s\n")

# Benchmarker qwen2.5:7b
print("3/3 Évaluation de qwen2.5:7b...")
result_qwen = benchmark_model(lm_ollama_qwen, 'qwen2.5:7b', valset, exact_match_metric)
results.append(result_qwen)
print(f"   Score: {result_qwen['score']:.2%} | Temps: {result_qwen['time']:.1f}s\n")

# Afficher le résumé
print("=" * 60)
print("📊 RÉSUMÉ DES PERFORMANCES")
print("=" * 60)
for r in sorted(results, key=lambda x: x['score'], reverse=True):
    print(f"{r['model']:20} | Score: {r['score']:6.2%} | Temps: {r['time']:5.1f}s")
print("=" * 60)

## 5.4 Architectures hybrides: utiliser différents modèles pour différentes tâches

Une **architecture hybride** utilise différents modèles pour différentes parties de votre pipeline. Par exemple :

- Modèle **rapide et économique** pour la catégorisation
- Modèle **précis mais coûteux** pour la priorisation

### Avantages

- 💰 **Optimisation des coûts** : Utiliser des modèles coûteux uniquement quand nécessaire
- ⚡ **Optimisation de la vitesse** : Modèles rapides pour les tâches simples
- 🎯 **Optimisation de la qualité** : Meilleurs modèles pour les tâches critiques

### Exemple: pipeline hybride

Créons un classifier qui utilise deux modèles différents :

In [ ]:
class HybridTicketClassifier(dspy.Module):
    """
    Classifier hybride utilisant 2 modèles différents:
    - Modèle rapide pour la catégorie
    - Modèle précis pour la priorité
    """
    
    def __init__(self, fast_lm, accurate_lm):
        super().__init__()
        self.fast_lm = fast_lm
        self.accurate_lm = accurate_lm
        
        # Signature pour la catégorie
        self.category_signature = CategoryClassifier
        
        # Signature pour la priorité
        self.priority_signature = PriorityClassifier
    
    def forward(self, ticket):
        # Étape 1: Catégorisation avec le modèle rapide
        with dspy.settings.context(lm=self.fast_lm):
            category_predictor = dspy.ChainOfThought(self.category_signature)
            category_result = category_predictor(ticket=ticket)
        
        # Étape 2: Priorisation avec le modèle précis
        with dspy.settings.context(lm=self.accurate_lm):
            priority_predictor = dspy.ChainOfThought(self.priority_signature)
            priority_result = priority_predictor(
                ticket=ticket,
                category=category_result.category
            )
        
        return dspy.Prediction(
            category=category_result.category,
            priority=priority_result.priority
        )

print("✅ Classe HybridTicketClassifier définie")

### Tester le classifier hybride

In [ ]:
print("🔀 Test du classifier hybride")
print("   Modèle rapide (catégorie): mistral:7b")
print("   Modèle précis (priorité): llama3.1:8b\n")

# Créer le classifier hybride
hybrid_classifier = HybridTicketClassifier(
    fast_lm=lm_ollama_mistral,      # Rapide pour la catégorie
    accurate_lm=lm_ollama_llama     # Précis pour la priorité
)

# Configurer DSPy (requis pour l'exécution)
dspy.configure(lm=lm_ollama_llama)

# Tester sur quelques exemples
print("📝 Exemples de prédictions:\n")

test_tickets = [
    "Mon ordinateur portable ne démarre plus, j'ai une présentation importante dans 2 heures",
    "Je voudrais accès au VPN pour le télétravail quand c'est possible",
    "Toutes les imprimantes de l'étage sont hors ligne"
]

for i, ticket in enumerate(test_tickets, 1):
    result = hybrid_classifier(ticket=ticket)
    print(f"{i}. Ticket: {ticket[:60]}...")
    print(f"   → Catégorie: {result.category} | Priorité: {result.priority}\n")

# Évaluer sur l'ensemble de validation
print("📊 Évaluation sur l'ensemble de validation:")
score_hybrid = evaluate_module(hybrid_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_hybrid:.2%}")

## 5.5 Guide de sélection de modèles

### Critères de sélection

| Critère | Ollama (local) | OpenAI | Anthropic |
|---------|---------------|--------|-----------|
| **Coût** | Gratuit (matériel local) | Payant à l'usage | Payant à l'usage |
| **Vitesse** | Dépend du matériel | Rapide (API cloud) | Rapide (API cloud) |
| **Confidentialité** | ✅ 100% local | ⚠️ Données envoyées à OpenAI | ⚠️ Données envoyées à Anthropic |
| **Qualité** | Bonne (7-8B params) | Excellente | Excellente |
| **Disponibilité** | Nécessite installation | API toujours disponible | API toujours disponible |
| **Latence** | Faible (local) | Moyenne (réseau) | Moyenne (réseau) |

### Recommandations par cas d'usage

#### 1. Développement et prototypage
**Choix recommandé** : Ollama (llama3.1:8b ou qwen2.5:7b)
- ✅ Gratuit, itérations rapides
- ✅ Pas de limite de requêtes
- ✅ Confidentialité des données

#### 2. Production à faible volume (<1000 requêtes/jour)
**Choix recommandé** : OpenAI (gpt-4o-mini) ou Anthropic (claude-3-5-haiku)
- ✅ Coûts acceptables
- ✅ Haute disponibilité
- ✅ Excellente qualité

#### 3. Production à haut volume (>10000 requêtes/jour)
**Choix recommandé** : Architecture hybride
- Ollama pour les tâches simples (catégorisation)
- API payante pour les tâches critiques (priorisation)
- ✅ Optimisation du rapport coût/performance

#### 4. Données sensibles (santé, finance, etc.)
**Choix recommandé** : Ollama uniquement
- ✅ Aucune donnée ne quitte votre infrastructure
- ✅ Conformité RGPD/HIPAA facilitée

#### 5. Recherche et expérimentation
**Choix recommandé** : Tous les modèles
- Tester plusieurs modèles pour trouver le meilleur
- Utiliser Ollama pour les itérations rapides
- Valider avec des modèles API avant production

## 5.6 Résumé de la Partie 5

### Ce que nous avons appris

1. **Abstraction DSPy** : Un seul code fonctionne avec tous les fournisseurs LLM
   
2. **Configuration de fournisseurs** :
   - **Ollama** : Local, gratuit, confidentialité maximale
   - **OpenAI** : API cloud, haute qualité, payant
   - **Anthropic** : API cloud, excellente qualité, payant

3. **Comparaison de modèles** :
   - Fonction de benchmarking pour comparer performances
   - Mesure du score ET du temps d'exécution
   - Aide à la décision data-driven

4. **Architectures hybrides** :
   - Différents modèles pour différentes tâches
   - Optimisation coût/performance/vitesse
   - Utilisation de `dspy.settings.context(lm=...)`

5. **Guide de sélection** :
   - Critères : coût, vitesse, confidentialité, qualité
   - Recommandations par cas d'usage
   - Stratégies adaptées au contexte

### Points clés à retenir

- ✅ **Flexibilité** : Changer de modèle ne nécessite que quelques lignes de code
- ✅ **Expérimentation** : Tester plusieurs modèles est facile et recommandé
- ✅ **Optimisation** : Architectures hybrides pour le meilleur rapport coût/performance
- ✅ **Confidentialité** : Ollama pour les données sensibles
- ✅ **Évolutivité** : Commencer avec Ollama, migrer vers API si nécessaire

### Exemple de workflow recommandé

```python
# 1. Développement avec Ollama (gratuit, rapide)
lm_dev = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434')
dspy.configure(lm=lm_dev)

# 2. Test avec plusieurs modèles
models = [lm_ollama_llama, lm_ollama_qwen, lm_openai_mini]
results = [benchmark_model(lm, name, valset, metric) for lm, name in models]

# 3. Production avec le meilleur modèle ou architecture hybride
lm_prod = best_model  # Ou HybridTicketClassifier(fast_lm, accurate_lm)
```

### Prochaines étapes

- **Partie 6** : Patterns avancés (optionnel - validation, retry, fallback)
- **Partie 7** : GEPA en pratique (optimisation sophistiquée)